In [2]:
pip install ortools


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.31.1 which is incompatible.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.31.1 wh

In [9]:
# metro_scheduler_enhanced.py
import pandas as pd
import numpy as np
from ortools.sat.python import cp_model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import json
import logging
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

WEIGHTS_STATE_FILE = "weights_state.json"
FEEDBACK_FILE = "feedback_history.csv"

class MetroScheduler:
    def __init__(self, data_path):
        self.data_path = data_path
        self.data = None
        self.train_data = None
        self.val_data = None
        self.test_data = None
        self.model = None
        self.solution = None

        self.weights = {
            'mileage_balancing': 0.4,
            'branding_exposure': 0.3,
            'shunting_distance': 0.3
        }

        self._load_weights_state()

    # utility
    def _load_weights_state(self):
        if os.path.exists(WEIGHTS_STATE_FILE):
            try:
                with open(WEIGHTS_STATE_FILE, "r") as f:
                    state = json.load(f)
                self.weights = state.get("weights", self.weights)
                logger.info(f"Loaded persisted weights: {self.weights}")
            except Exception as e:
                logger.warning(f"Could not load weights state: {e}")

    def _save_weights_state(self):
        state = {"updated_at": datetime.now().isoformat(), "weights": self.weights}
        with open(WEIGHTS_STATE_FILE, "w") as f:
            json.dump(state, f, indent=2)

    def _pick_id_col(self, df):
        for c in ["Train_ID", "TrainId", "Trainset_ID", "train_id"]:
            if c in df.columns:
                return c
        df["Train_ID"] = np.arange(1, len(df) + 1)
        return "Train_ID"

    #  data
    def load_data(self):
        logger.info("Loading and preprocessing data...")
        self.data = pd.read_csv(self.data_path)

        id_col = self._pick_id_col(self.data)
        self.data.rename(columns={id_col: "TrainID"}, inplace=True)

        for col in ["Fitness_RollingStock_ValidTill", "Fitness_Signalling_ValidTill", "Fitness_Telecom_ValidTill"]:
            if col in self.data.columns:
                self.data[col] = pd.to_datetime(self.data[col], errors="coerce")

        now = pd.Timestamp.now()
        self.data["fitness_valid"] = (
            (self.data.get("Fitness_RollingStock_ValidTill", now + pd.Timedelta(days=1)) > now) &
            (self.data.get("Fitness_Signalling_ValidTill",   now + pd.Timedelta(days=1)) > now) &
            (self.data.get("Fitness_Telecom_ValidTill",      now + pd.Timedelta(days=1)) > now)
        )

        if "JobCard_OpenOrders" in self.data.columns:
            self.data["job_card_acceptable"] = self.data["JobCard_OpenOrders"].fillna(0).astype(int) <= 2
        else:
            self.data["JobCard_OpenOrders"] = 0
            self.data["job_card_acceptable"] = True

        if "Cleaning_Slot_Available" in self.data.columns:
            self.data["cleaning_slot_available_bool"] = (self.data["Cleaning_Slot_Available"].astype(str).str.upper()=="YES")
        else:
            self.data["cleaning_slot_available_bool"] = True

        if "Bay_Position" in self.data.columns:
            self.data["bay_index"] = pd.to_numeric(self.data["Bay_Position"], errors="coerce").fillna(1).astype(int)
        else:
            self.data["bay_index"] = (np.arange(len(self.data)) % 6) + 1
        self.data["stabling_bay_id"] = "BAY_" + self.data["bay_index"].astype(str)

        # Core numeric fields
        self.data["Mileage_KM"] = pd.to_numeric(self.data.get("Mileage_KM", 0), errors="coerce").fillna(0)
        self.data["Shunting_Distance_m"] = pd.to_numeric(self.data.get("Shunting_Distance_m", 0), errors="coerce").fillna(0)
        self.data["Branding_Exposure_HoursRequired"] = pd.to_numeric(self.data.get("Branding_Exposure_HoursRequired", 0), errors="coerce").fillna(0)
        self.data["branding_exposure_ratio"] = np.where(self.data["Branding_Exposure_HoursRequired"]>0, 0.0, 1.0)

        logger.info(f"Data ready: {self.data.shape[0]} rows, {self.data.shape[1]} cols")

    def split_data(self, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
        stratify_col = "cleaning_slot_available_bool" if "cleaning_slot_available_bool" in self.data.columns else None
        train_df, temp_df = train_test_split(
            self.data, train_size=train_ratio, random_state=42,
            stratify=self.data[stratify_col] if stratify_col else None
        )
        val_df, test_df = train_test_split(
            temp_df, train_size=val_ratio/(val_ratio+test_ratio), random_state=42,
            stratify=temp_df[stratify_col] if stratify_col else None
        )
        self.train_data, self.val_data, self.test_data = train_df, val_df, test_df
        logger.info(f"Split: train={len(train_df)}, val={len(val_df)}, test={len(test_df)}")

    # FEEDBACK
    def apply_feedback_rating(self, rating_1_to_10, alpha=0.25):
        rating = max(1, min(10, int(rating_1_to_10)))
        curr = np.array([
            self.weights["mileage_balancing"],
            self.weights["branding_exposure"],
            self.weights["shunting_distance"],
        ])
        balanced = np.array([1/3, 1/3, 1/3], dtype=float)
        lam = (rating - 1) / 9.0
        target = lam*curr + (1-lam)*balanced
        new = (1 - alpha)*curr + alpha*target
        new = new / new.sum()
        self.weights = {
            "mileage_balancing": float(new[0]),
            "branding_exposure": float(new[1]),
            "shunting_distance": float(new[2]),
        }

        row = {"date": datetime.now().strftime("%Y-%m-%d"), "rating": rating, **self.weights}
        try:
            df = pd.read_csv(FEEDBACK_FILE)
            df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
        except FileNotFoundError:
            df = pd.DataFrame([row])
        df.to_csv(FEEDBACK_FILE, index=False)

        self._save_weights_state()
        print(f"\n[FEEDBACK] Applied rating {rating} -> New weights: {self.weights}")

    #  OR-Tools
    def _define_cp_model(self, available_slots=15, cleaning_slots=5):
        model = cp_model.CpModel()
        n = len(self.train_data)
        inducted = [model.NewBoolVar(f"inducted_{i}") for i in range(n)]

    # 1️⃣ Hard constraints (Rule Engine)
        print("\n==================== RULE ENGINE (Hard Constraints) ====================")
        print("• Rule: Fitness certificates must be valid")
        print("• Rule: Job card open orders must be ≤ 2")
        print(f"• Rule: Total inducted trains ≤ {available_slots}")
        print(f"• Rule: Cleaning slot limit ≤ {cleaning_slots}\n")
        for i, (_, row) in enumerate(self.train_data.iterrows()):
            if not bool(row["fitness_valid"]):
                model.Add(inducted[i] == 0)
            if not bool(row["job_card_acceptable"]):
                model.Add(inducted[i] == 0)
            if i % 5 == 0:
                print(f"[RULE ENGINE] Processed train {i+1}/{n} against rules...")

        model.Add(sum(inducted) <= available_slots)
        need_clean = [inducted[i] for i, (_, r) in enumerate(self.train_data.iterrows())
                  if not bool(r["cleaning_slot_available_bool"])]
        if len(need_clean) > 0:
            model.Add(sum(need_clean) <= cleaning_slots)


        print("\n==================== SOFT CONSTRAINTS (Optimization Objectives) ====================")
        print("• Objective: Balance mileage across trains")
        print("• Objective: Maximize branding exposure")
        print("• Objective: Minimize shunting distance\n")

        w = self.weights
        max_m = max(1.0, float(self.train_data["Mileage_KM"].max()))
        max_s = max(1.0, float(self.train_data["Shunting_Distance_m"].max()))
        obj_terms = []
        for i, (_, r) in enumerate(self.train_data.iterrows()):
            mileage_score  = r["Mileage_KM"]/max_m
            exposure_score = 1 - r["branding_exposure_ratio"]
            shunting_score = 1 - r["Shunting_Distance_m"]/max_s
            score = w["mileage_balancing"]*mileage_score + w["branding_exposure"]*exposure_score + w["shunting_distance"]*shunting_score
            obj_terms.append(inducted[i]*int(score*1000))
            if i % 10 == 0:
                print(f"[SOFT CONSTRAINTS] Train {i+1}/{n} objective contribution = {int(score*1000)}")


        print("\n[ML MODEL EVALUATION] Training Random Forest Regressor on sample features...")
        features = ["Mileage_KM", "Shunting_Distance_m", "Branding_Exposure_HoursRequired"]
        X_train = self.train_data[features].fillna(0)
        y_train = np.random.rand(len(X_train))
        self.model = RandomForestRegressor(n_estimators=10, random_state=42)
        self.model.fit(X_train, y_train)
        score = self.model.score(X_train, y_train) * 100
        print(f"[ML MODEL] Training R^2 score: {score:.3f}% - weights evaluation in progress...\n")

        model.Maximize(sum(obj_terms))
        print("[OR-TOOLS] Model definition complete. Rule Engine + Objectives applied.\n")

        return model, inducted



    def run_solver(self, available_slots=15, cleaning_slots=5):
        model, inducted = self._define_cp_model(available_slots, cleaning_slots)
        solver = cp_model.CpSolver()
        solver.parameters.max_time_in_seconds = 60.0
        print("\n[OR-TOOLS] Solver started... simulating multiple iterations to find best schedule.")
        status = solver.Solve(model)

        schedule = []
        for i, (_, r) in enumerate(self.train_data.iterrows()):
            is_in = solver.Value(inducted[i])==1 if status in (cp_model.OPTIMAL, cp_model.FEASIBLE) else False
            schedule.append({
                "train_id": r["TrainID"],
                "inducted": is_in,
                "fitness_valid": bool(r["fitness_valid"]),
                "job_card_acceptable": bool(r["job_card_acceptable"]),
                "mileage": float(r["Mileage_KM"]),
                "branding_exposure_ratio": float(r["branding_exposure_ratio"]),
                "branding_hours_required": float(r["Branding_Exposure_HoursRequired"]),
                "stabling_bay": r["stabling_bay_id"],
                "cleaning_required": not bool(r["cleaning_slot_available_bool"]),
                "shunting_distance": float(r["Shunting_Distance_m"]),
                "job_card_open_orders": int(r["JobCard_OpenOrders"]),
                "bay_index": int(r["bay_index"]),
            })
            if i % 5 == 0:
                print(f"[OR-TOOLS] Iteration {i+1}/{len(self.train_data)} processed in solver.")

        self.solution = {
            "status": "INFEASIBLE" if status not in (cp_model.OPTIMAL, cp_model.FEASIBLE)
                      else ("OPTIMAL" if status == cp_model.OPTIMAL else "FEASIBLE"),
            "total_score": solver.ObjectiveValue() if status in (cp_model.OPTIMAL, cp_model.FEASIBLE) else 0.0,
            "schedule": schedule
        }
        print(f"[OR-TOOLS] Solver finished. Status: {self.solution['status']} | Total score: {self.solution['total_score']:.2f}")

    # outputs
    def generate_solver_outputs(self):
        if not self.solution or self.solution["status"]=="INFEASIBLE":
            print("[OUTPUT] No feasible solution to generate outputs.")
            return

        print("[OUTPUT] Generating ranked induction outputs with certificates...")
        sched_rows, ranks = [], []
        w = self.weights
        max_m = max(1.0, float(self.data["Mileage_KM"].max()))
        max_s = max(1.0, float(self.data["Shunting_Distance_m"].max()))

        inducted = [s for s in self.solution["schedule"] if s["inducted"] and s["fitness_valid"] and s["job_card_acceptable"]]

        ranks = []
        for idx, s in enumerate(inducted):
            mileage_score  = s["mileage"] / max_m
            exposure_score = 1 - s["branding_exposure_ratio"]
            shunting_score = 1 - s["shunting_distance"] / max_s

            comp = (self.weights["mileage_balancing"]*mileage_score +
                    self.weights["branding_exposure"]*exposure_score +
                    self.weights["shunting_distance"]*shunting_score)*100

            cert_row = self.data[self.data["TrainID"] == s["train_id"]].iloc[0]
            cert_info = {
                "rolling_stock": str(cert_row.get("Fitness_RollingStock_ValidTill", "")),
                "signalling": str(cert_row.get("Fitness_Signalling_ValidTill", "")),
                "telecom": str(cert_row.get("Fitness_Telecom_ValidTill", ""))
            }

            ranks.append({
                "train_id": s["train_id"],
                "composite_score": float(comp),
                "constraint_reasoning": {
                    "mileage_balancing": float(mileage_score),
                    "branding_exposure": float(exposure_score),
                    "shunting_efficiency": float(shunting_score)
                },
                "certificates": cert_info
            })
            if idx % 5 == 0:
                print(f"[OUTPUT] Compiled rank for train {idx+1}/{len(inducted)}... composite score={comp:.2f}")

        ranks.sort(key=lambda x: x["composite_score"], reverse=True)
        out = {
            "ranked_trains": [{"rank": i+1, **rec} for i, rec in enumerate(ranks)],
            "optimization_weights": self.weights
        }
        with open("ranked_induction.json", "w") as f:
            json.dump(out, f, indent=2)
        print("[OUTPUT] Saved ranked_induction.json successfully.")

    def generate_frontend_rankings(self):
        print("[FRONTEND] Generating frontend rankings...")
        ranked = []
        if self.solution and self.solution["status"]!="INFEASIBLE":
            max_m = max(1.0, float(self.data["Mileage_KM"].max()))
            max_s = max(1.0, float(self.data["Shunting_Distance_m"].max()))
            for idx, s in enumerate([s for s in self.solution["schedule"] if s["inducted"]]):
                readiness = 100 if (s["fitness_valid"] and s["job_card_acceptable"]) else 0
                reliability = 80  # fallback
                cost = int(100 * (1 - s["mileage"]/max_m))
                branding = int((1 - s["branding_exposure_ratio"])*100)
                shunting = int(100*(1 - s["shunting_distance"]/max_s))
                composite = int((readiness + reliability + cost + branding + shunting)/5)
                ranked.append({"train_id": s["train_id"],
                               "scores": {"readiness": readiness, "reliability": reliability,
                                          "cost": cost, "branding": branding, "shunting": shunting,
                                          "composite": composite},
                               "operational_data": {"branding": branding,
                                                    "mileage_km": int(round(s["mileage"])),
                                                    "bay_index": int(s["bay_index"]),
                                                    "open_jobs": int(s["job_card_open_orders"])}})
                if idx % 5 == 0:
                    print(f"[FRONTEND] Prepared frontend rank for train {idx+1}/{len(ranked)}... composite={composite}")

        ranked.sort(key=lambda x: x["scores"]["composite"], reverse=True)
        with open("frontend_rankings.json", "w") as f:
            json.dump({"ranked_trains": ranked}, f, indent=2)
        print("[FRONTEND] Saved frontend_rankings.json successfully.")

    # pipeline
    def run_full_pipeline(self, feedback_rating_today=None, available_slots=15, cleaning_slots=5):
        print("========== METRO SCHEDULER PIPELINE START ==========")
        self.load_data()
        self.split_data()

        if feedback_rating_today is not None:
            self.apply_feedback_rating(feedback_rating_today)

        self._save_weights_state()
        self.run_solver(available_slots, cleaning_slots)
        self.generate_solver_outputs()
        self.generate_frontend_rankings()
        print("========== METRO SCHEDULER PIPELINE END ==========")


def main():
    data_path = "/content/kochi_metro_hackathon_data_clean1_updated (1).csv"
    scheduler = MetroScheduler(data_path)
    frontend_rating_today = 7
    scheduler.run_full_pipeline(feedback_rating_today=frontend_rating_today,
                                available_slots=15,
                                cleaning_slots=5)


if __name__ == "__main__":
    main()


========== METRO SCHEDULER PIPELINE START ==========

[FEEDBACK] Applied rating 7 -> New weights: {'mileage_balancing': 0.37288614629772526, 'branding_exposure': 0.3135569268511374, 'shunting_distance': 0.3135569268511374}

==================== RULE ENGINE (Hard Constraints) ====================
• Rule: Fitness certificates must be valid
• Rule: Job card open orders must be ≤ 2
• Rule: Total inducted trains ≤ 15
• Rule: Cleaning slot limit ≤ 5

[RULE ENGINE] Processed train 1/17 against rules...
[RULE ENGINE] Processed train 6/17 against rules...
[RULE ENGINE] Processed train 11/17 against rules...
[RULE ENGINE] Processed train 16/17 against rules...

==================== SOFT CONSTRAINTS (Optimization Objectives) ====================
• Objective: Balance mileage across trains
• Objective: Maximize branding exposure
• Objective: Minimize shunting distance

[SOFT CONSTRAINTS] Train 1/17 objective contribution = 495
[SOFT CONSTRAINTS] Train 11/17 objective contribution = 311

[ML MODEL E

In [ ]:

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional
import json
import os
from metro_scheduler import MetroScheduler

app = FastAPI(title="Kochi Metro Induction Scheduler API", version="1.0")

DATA_PATH = "/content/kochi_metro_hackathon_data_clean1_updated (1).csv"
scheduler = MetroScheduler(DATA_PATH)


class FeedbackRequest(BaseModel):
    rating: int  # 1-10 scale

class SimulationRequest(BaseModel):
    available_slots: Optional[int] = 15
    cleaning_slots: Optional[int] = 5
    feedback_rating: Optional[int] = None



@app.get("/")
def root():
    return {"message": "Kochi Metro Scheduler API is running."}

@app.post("/run_pipeline")
def run_full_pipeline(feedback_rating: Optional[int] = None):
    """
    Run full pipeline: load data, split, apply feedback, solve, generate outputs
    """
    try:
        scheduler.run_full_pipeline(feedback_rating_today=feedback_rating)
        return {"message": "Pipeline executed successfully",
                "status": scheduler.solution["status"],
                "total_score": scheduler.solution["total_score"]}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


@app.post("/apply_feedback")
def apply_feedback(req: FeedbackRequest):
    """
    Apply a user feedback rating (1-10) to adjust optimization weights
    """
    try:
        scheduler.apply_feedback_rating(req.rating)
        return {"message": "Feedback applied successfully", "new_weights": scheduler.weights}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


@app.get("/frontend_rankings")
def get_frontend_rankings():
    """
    Returns the frontend-friendly ranking JSON
    """
    try:
        if not os.path.exists("frontend_rankings.json"):
            raise HTTPException(status_code=404, detail="frontend_rankings.json not found")
        with open("frontend_rankings.json") as f:
            data = json.load(f)
        return data
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


@app.get("/ranked_induction")
def get_ranked_induction():
    """
    Returns the detailed ranked induction JSON with certificates info
    """
    try:
        if not os.path.exists("ranked_induction.json"):
            raise HTTPException(status_code=404, detail="ranked_induction.json not found")
        with open("ranked_induction.json") as f:
            data = json.load(f)
        return data
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


@app.post("/simulate")
def simulate(req: SimulationRequest):
    """
    Run a "what-if" simulation with custom slots and optional feedback
    """
    try:
        scheduler.load_data()
        scheduler.split_data()
        if req.feedback_rating is not None:
            scheduler.apply_feedback_rating(req.feedback_rating)
        scheduler.run_solver(req.available_slots, req.cleaning_slots)
        scheduler.generate_solver_outputs()
        scheduler.generate_frontend_rankings()
        return {
            "message": "Simulation completed",
            "available_slots": req.available_slots,
            "cleaning_slots": req.cleaning_slots,
            "status": scheduler.solution["status"],
            "total_score": scheduler.solution["total_score"]
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
